In [1]:
import xgboost as xgb
import pandas as pd
import numpy as np
import random

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
alltrain_df = pd.read_csv('./all_train_nohash.csv')
alltest_df = pd.read_csv('./all_test_nohash.csv')

/usr/cac/rhel6/lsa/anaconda-vis-portal/latest/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (28,59,60,68) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/cac/rhel6/lsa/anaconda-vis-portal/latest/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (27,66) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
drops = ["Unnamed: 0",
         "VE_FORMS.y","HARM_EV.y","MAN_COLL.y","RAIL","TWAY_ID",
         "CF1","CF2","CF3",
         "VIN","VIN_1","VIN_2","VIN_3","VIN_4","VIN_5","VIN_6","VIN_7","VIN_8",
         "VIN_9","VIN_10","VIN_11","VIN_12","MCARR_ID",
         "VEH_NO.y","COUNTY.y", "DAY.y","MONTH.y","HOUR.y","MINUTE.y","ROAD_FNC.y",
         "SCH_BUS.y","MAKE.y","MAK_MOD.y","BODY_TYP.y","MOD_YEAR.y",
         "TOW_VEH.y","SPEC_USE.y","EMER_USE.y","ROLLOVER.y","IMPACT1.y","FIRE_EXP.y",
         "CERT_NO"
        ]

In [5]:
for col in drops:
    del alltrain_df[col]

In [6]:
for col in drops:
    del alltest_df[col]

In [7]:
train_x = alltrain_df.drop(['DRUNK_DR','YEAR'], axis=1)
train_y = alltrain_df['DRUNK_DR']
test_x = alltest_df

In [8]:
trainX = train_x.as_matrix()
trainY = train_y.astype(int)
testX = test_x.as_matrix()

In [9]:
dtrain = xgb.DMatrix(trainX, label=trainY)
dtest = xgb.DMatrix(testX)

### Parameter tuning
1. for `scale_pos_weight`, read more [here](https://github.com/dmlc/xgboost/blob/master/demo/kaggle-higgs/speedtest.py)
2. If you care about predicting the right probability, which in such a case, you cannot re-balance the dataset. Set parameter `max_delta_step` to a finite number (say 1) will help convergence. Read more [here](https://github.com/dmlc/xgboost/blob/master/doc/param_tuning.md)

In [19]:
# parameter tuning
# random.seed(1024)

param = {}
param['objective'] = 'binary:logitraw'

# scale weight of positive examples
# rescale weight to make it same as test set
# testsize = 543015
# weight = dtrain[:,31] * float(test_size) / len(train_y)
# param['scale_pos_weight'] = sum_wneg/sum_wpos

ratio = float(np.sum(train_y == 1)) / np.sum(train_y == 0)
param['scale_pos_weight'] = ratio

param['eta'] = 0.05
param['max_depth'] = 7
param['eval_metric'] = 'auc'
# param['silent'] = 1
param['min_child_weight'] = 1
param['subsample'] = 0.7
param['colsample_bytree'] = 0.7
param['nthread'] = 8

# Sean:: can't understand how to get dtrain[:, 31]=weight !!
# def fpreproc(dtrain, dtest, param):
#     label = dtrain.get_label()
#     ratio = float(np.sum(label == 0)) / np.sum(label==1)
#     print("ratio= ", ratio)
#     param['scale_pos_weight'] = ratio
#     wtrain = dtrain.get_weight()
#     wtest = dtest.get_weight()
#     sum_weight = sum(wtrain) + sum(wtest)
#     wtrain *= sum_weight / sum(wtrain)
#     wtest *= sum_weight / sum(wtest)
#     print("final wtrain, wtest = ", wtrain, wtest)
#     dtrain.set_weight(wtrain)
#     dtest.set_weight(wtest)
#     return (dtrain, dtest, param)

# XGBoostError: b'[13:07:13] src/metric/rank_metric.cc:36: 
# Check failed: (info.weights.size()) == (ndata) we need weight to evaluate ams'
#
# bst_cv = xgb.cv(param, dtrain, num_boost_round=50, nfold=5, metrics={'ams@0.15', 'auc'}, seed=0)
# bst_cv

In [20]:
bst_cv = xgb.cv(param, dtrain, num_boost_round=1000, nfold=5, seed=0)
bst_cv

,test-auc-mean,test-auc-std,train-auc-mean,train-auc-std
0,0.809241,0.000558,0.809489,0.000430
1,0.816964,0.001739,0.817245,0.001386
2,0.820000,0.001534,0.820423,0.001988
3,0.822810,0.001110,0.823292,0.001050
4,0.823983,0.000944,0.824446,0.001291
5,0.824637,0.001015,0.825124,0.001683
6,0.826328,0.000584,0.826725,0.001356
7,0.826693,0.000427,0.827116,0.001099
8,0.826891,0.000742,0.827355,0.001090
9,0.827419,0.000920,0.827887,0.001409


In [42]:
tst = pd.DataFrame(bst_cv)
test = tst['train-auc-mean']-tst['train-auc-std']
test.idxmax()

49

In [26]:
num_round = test.idxmax()

In [27]:
bst = xgb.train(param, dtrain, num_round)
bst

In [28]:
ypred = bst.predict(dtest)

### Given a prediction, create a Kaggle submission file


In [47]:
submit = pd.DataFrame(data={'ID': alltest_df['ID'], 'DRUNK_DR': ypred})

,DRUNK_DR,ID
0,-1.183980,0
1,-1.499542,1
2,-0.608850,1
3,-1.518499,1
4,-1.572906,1
5,-0.547461,1
6,-1.631071,1
7,-2.236388,2
8,-2.535650,2
9,-2.390640,2


In [49]:
submit.to_csv('fars_submit_xgb003_production.csv', index = False)